In [1]:
import pandas as pd
import requests
import boto3
import ast
import datetime
import time

In [3]:
from cobinhood_api import Cobinhood
cob = Cobinhood()

print(cob.system.get_time())

2018-08-14 06:12:02 Chriss-MacBook-Pro.local cobinhood-api[1032] DEBUG cobinhood_api.http.system fetch "get_time"


{'success': True, 'result': {'time': 1534241522718}}


In [3]:
cob.market.get_trading_pairs()

2018-07-27 07:16:56 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'success': True,
 'result': {'trading_pairs': [{'id': 'ABT-BTC',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'ABT-ETH',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'ETH',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.0000001',
    'margin_enabled': False},
   {'id': 'ACT-BTC',
    'base_currency_id': 'ACT',
    'quote_currency_id': 'BTC',
    'base_max_size': '11934470.211',
    'base_min_size': '238.69',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-BTC',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2991280.418',
    'base_min_size': '59.826',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-ETH',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'ETH',

In [4]:
# Instantiate resources
try:
    boto_session = boto3.Session(profile_name='loidsig')
except:
    boto_session = boto3.Session()
s3_resource = boto_session.resource('s3')
s3_bucket = 'loidsig-crypto'

sm_client = boto_session.client(
    service_name='secretsmanager',
    region_name='us-east-1',
    endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
)
get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_CPM_Cobinhood')
key, value = ast.literal_eval(get_secret_value_response['SecretString']).popitem()
cob = Cobinhood(API_TOKEN=value)

In [5]:
print(cob.wallet.get_balances())

2018-08-14 06:12:23 Chriss-MacBook-Pro.local cobinhood-api[1032] DEBUG cobinhood_api.http.wallet fetch "get_balances"


{'success': True, 'result': {'balances': [{'currency': 'PLY', 'type': 'exchange', 'total': '2', 'on_order': '0', 'locked': False, 'usd_value': '0.231544796', 'btc_value': '0.00003836'}, {'currency': 'SHPING', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.000482888', 'btc_value': '0.00000008'}, {'currency': 'HQX', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.008035278', 'btc_value': '0.00000133'}, {'currency': 'FXT', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.03199133', 'btc_value': '0.0000053'}, {'currency': 'USDT', 'type': 'exchange', 'total': '2.4501111324', 'on_order': '0', 'locked': False, 'usd_value': '2.46005160715110560424', 'btc_value': '0.000406448935753836'}, {'currency': 'ETH', 'type': 'exchange', 'total': '1.48', 'on_order': '0', 'locked': False, 'usd_value': '395.04756842856', 'btc_value': '0.0652715224'}]}}


### Get usd wallet balance

In [11]:
wallet_dict = cob.wallet.get_balances()['result']['balances']
wallet_df = pd.DataFrame(wallet_dict)
wallet_df.rename(columns={'total':'quantity', 'currency':'coin'}, inplace=True)
wallet_df['access_datetime'] = datetime.datetime.utcfromtimestamp(time.time()).strftime('%Y-%m-%d %I:%M:%S')
wallet_df

2018-08-14 06:21:10 Chriss-MacBook-Pro.local cobinhood-api[1032] DEBUG cobinhood_api.http.wallet fetch "get_balances"


,btc_value,coin,locked,on_order,quantity,type,usd_value,access_datetime
0,0.00003836,PLY,False,0,2,exchange,0.23257668,2018-08-14 10:21:10
1,0.00000008,SHPING,False,0,1,exchange,0.00048504,2018-08-14 10:21:10
2,0.00000133,HQX,False,0,1,exchange,0.0080973225,2018-08-14 10:21:10
3,0.0000053,FXT,False,0,1,exchange,0.0321339,2018-08-14 10:21:10
4,0.000404292837957324,USDT,False,0,2.4501111324,exchange,2.45326382490516024876,2018-08-14 10:21:10
5,0.0654283728,ETH,False,0,1.48,exchange,397.02444265548,2018-08-14 10:21:10


In [16]:
from sqlalchemy import create_engine
def logic_db_engine():
    """Fetches Logic DB postgres connection object

    Returns:
        A database connection object for Postgres
    """
    try:
        boto_session = boto3.Session(profile_name='loidsig')
    except:
        boto_session = boto3.Session()
    sm_client = boto_session.client(
        service_name='secretsmanager',
        region_name='us-east-1',
        endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
    )
    get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_DB')
    cred_dict = ast.literal_eval(get_secret_value_response['SecretString'])
    db_user, db_pass = cred_dict['username'], cred_dict['password']
    db_host, db_port, db_name = cred_dict['host'], cred_dict['port'], cred_dict['dbname']
    print(db_user)

    try:
        postgres_engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')
    except Exception as e:
        print("Unable to connect to postgres! Error: {}".format(e))
        raise
    return postgres_engine

In [18]:
wallet_df.to_sql('cobinhood', logic_db_engine(), schema='wallet')

the_logic


### Candlestick data

In [22]:
cob_candles_list = cob.chart.get_candles(trading_pair_id='ETH-USDT', start_time=1519862400000, end_time=1519948800000, timeframe='1m')['result']['candles']
df = pd.DataFrame(cob_candles_list)
df.rename(columns={'trading_pair_id':'coin','timestamp':'close_timestamp'}, inplace=True)
df['coin'] = df['coin'].replace(regex=True, to_replace=r'-', value=r'')
df['close_timestamp_trim'] = df['close_timestamp']/1000
df['close_datetime'] = pd.to_datetime(df['close_timestamp_trim'], unit='s')

print(f"""Current: {datetime.datetime.utcfromtimestamp(time.time()).strftime('%Y-%m-%d %I:%M:%S')}
Start: {datetime.datetime.utcfromtimestamp(1519516800).strftime('%Y-%m-%d %I:%M:%S')}""")
df

2018-07-27 07:56:21 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.chart fetch "get_candles"


Current: 2018-07-27 11:56:22
Start: 2018-02-25 12:00:00


,close,high,low,open,timeframe,close_timestamp,coin,volume,close_timestamp_trim,close_datetime
0,479.19,479.19,477.77,477.77,1m,1532606160000,ETHUSDT,0.09,1.532606e+09,2018-07-26 11:56:00
1,479.19,479.19,479.19,479.19,1m,1532606220000,ETHUSDT,0,1.532606e+09,2018-07-26 11:57:00
2,478.81,478.81,477.5,477.5,1m,1532606280000,ETHUSDT,2.91145321290685,1.532606e+09,2018-07-26 11:58:00
3,478.81,478.81,478.81,478.81,1m,1532606340000,ETHUSDT,0,1.532606e+09,2018-07-26 11:59:00
4,478.02,478.34,478.02,478.34,1m,1532606400000,ETHUSDT,2.263513507483162,1.532606e+09,2018-07-26 12:00:00
5,477.51,477.51,477.51,477.51,1m,1532606460000,ETHUSDT,0.048,1.532606e+09,2018-07-26 12:01:00
6,479.14,479.14,479.14,479.14,1m,1532606520000,ETHUSDT,0.04783671,1.532607e+09,2018-07-26 12:02:00
7,479.14,479.14,479.14,479.14,1m,1532606580000,ETHUSDT,0,1.532607e+09,2018-07-26 12:03:00
8,479.14,479.14,479.14,479.14,1m,1532606640000,ETHUSDT,0,1.532607e+09,2018-07-26 12:04:00
9,479.14,479.14,479.14,479.14,1m,1532606700000,ETHUSDT,0,1.532607e+09,2018-07-26 12:05:00


In [21]:
df[df['volume'] != '0']

,close,high,low,open,timeframe,close_timestamp,coin,volume,close_timestamp_trim,close_datetime
132,879.9,879.9,870,870,1m,1519870320000,ETHUSDT,1.12,1.519870e+09,2018-03-01 02:12:00
156,899.9,899.9,879.9,879.9,1m,1519871760000,ETHUSDT,0.6469286,1.519872e+09,2018-03-01 02:36:00
191,899.9,899.9,899.9,899.9,1m,1519873860000,ETHUSDT,0.31932385,1.519874e+09,2018-03-01 03:11:00
315,840,855.1,840,855.1,1m,1519881300000,ETHUSDT,1,1.519881e+09,2018-03-01 05:15:00
321,838.15,840,838.15,840,1m,1519881660000,ETHUSDT,1,1.519882e+09,2018-03-01 05:21:00
324,825,838.15,825,838.15,1m,1519881840000,ETHUSDT,1.48,1.519882e+09,2018-03-01 05:24:00
647,825,836.1,825,836.1,1m,1519901220000,ETHUSDT,1,1.519901e+09,2018-03-01 10:47:00
880,875,875,875,875,1m,1519915200000,ETHUSDT,0.05991693,1.519915e+09,2018-03-01 14:40:00
881,898.8,898.8,898.8,898.8,1m,1519915260000,ETHUSDT,0.04792709,1.519915e+09,2018-03-01 14:41:00
901,840,840,840,840,1m,1519916460000,ETHUSDT,0.098,1.519916e+09,2018-03-01 15:01:00


In [2]:
'eth-usdt'.replace('-', '')

'ethusdt'

### Placing an order

In [20]:
data = {
    "trading_pair_id": "ETH-USDT",
    "side": "bid",
    "type": "market",
    "size": "1.000"
}
result = cob.trading.post_orders(data)

2018-08-06 08:02:53 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.trading fetch "post_orders"


In [21]:
trade_id = result['result']['order']['id']
trade_id

'7b03bbf4-e67c-44ea-878c-11fe822462f0'

In [22]:
cob.trading.get_orders(trade_id)

2018-08-06 08:02:59 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.trading fetch "get_orders"


{'success': True,
 'result': {'order': {'id': '7b03bbf4-e67c-44ea-878c-11fe822462f0',
   'trading_pair_id': 'ETH-USDT',
   'side': 'bid',
   'type': 'market',
   'price': '0',
   'size': '1',
   'filled': '1',
   'state': 'filled',
   'timestamp': 1533556974069,
   'eq_price': '408.0537288676',
   'completed_at': '2018-08-06T12:02:54.095305855Z',
   'source': 'exchange'}}}

In [23]:
print(cob.wallet.get_balances())

2018-08-06 08:03:14 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.wallet fetch "get_balances"


{'success': True, 'result': {'balances': [{'currency': 'USDT', 'type': 'exchange', 'total': '2.4501111324', 'on_order': '0', 'locked': False, 'usd_value': '2.4511962633444842022', 'btc_value': '0.000351002920827624'}, {'currency': 'ETH', 'type': 'exchange', 'total': '1.48', 'on_order': '0', 'locked': False, 'usd_value': '606.16495162776', 'btc_value': '0.0867979744'}]}}


### Other Data

In [27]:
cob.market.get_currencies()

2018-08-10 07:13:37 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_currencies"


{'success': True,
 'result': {'currencies': [{'currency': 'COB',
    'name': 'COBINHOOD Token',
    'type': 'erc20',
    'min_unit': '0.00000001',
    'deposit_fee': '0',
    'withdrawal_fee': '86.15',
    'min_withdrawal': '287.15392',
    'funding_min_size': '861.462',
    'interest_increment': '0.0001',
    'is_visible': True,
    'funding_enabled': True,
    'quotation_enabled': False,
    'required_kyc_level': 1,
    'deposit_frozen': False,
    'withdrawal_frozen': False,
    'cob_withdrawal_fee': '0'},
   {'currency': 'COFI',
    'name': 'CoinFi',
    'type': 'erc20',
    'min_unit': '0.00000001',
    'deposit_fee': '0',
    'withdrawal_fee': '144.15',
    'min_withdrawal': '480.49023',
    'funding_min_size': '1441.471',
    'interest_increment': '0.0001',
    'is_visible': True,
    'funding_enabled': False,
    'quotation_enabled': False,
    'required_kyc_level': 1,
    'deposit_frozen': False,
    'withdrawal_frozen': False,
    'cob_withdrawal_fee': '41.32972028'},
   {'cu

In [30]:
pairs = cob.market.get_trading_pairs()
pairs

2018-08-10 07:15:21 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'success': True,
 'result': {'trading_pairs': [{'id': 'ABT-BTC',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'BTC',
    'base_max_size': '4233908.184',
    'base_min_size': '84.679',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'ABT-ETH',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'ETH',
    'base_max_size': '4233908.184',
    'base_min_size': '84.679',
    'quote_increment': '0.0000001',
    'margin_enabled': False},
   {'id': 'ACT-BTC',
    'base_currency_id': 'ACT',
    'quote_currency_id': 'BTC',
    'base_max_size': '20444594.49',
    'base_min_size': '408.892',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-BTC',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'BTC',
    'base_max_size': '4063529.991',
    'base_min_size': '81.271',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-ETH',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'ETH',

In [36]:
pairs_payload = cob.market.get_trading_pairs()
pairs_list = [[pair['id'].replace('-', ''), pair['id']] for pair in pairs_payload['result']['trading_pairs']]
pairs_dict = dict(pairs_list)
pairs_dict

2018-08-10 07:21:39 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'ABTBTC': 'ABT-BTC',
 'ABTETH': 'ABT-ETH',
 'ACTBTC': 'ACT-BTC',
 'BATBTC': 'BAT-BTC',
 'BATETH': 'BAT-ETH',
 'BCHBTC': 'BCH-BTC',
 'BCHETH': 'BCH-ETH',
 'BCHUSDT': 'BCH-USDT',
 'BDGBTC': 'BDG-BTC',
 'BDGETH': 'BDG-ETH',
 'BDGUSDT': 'BDG-USDT',
 'BEEBTC': 'BEE-BTC',
 'BEEETH': 'BEE-ETH',
 'BMBTC': 'BM-BTC',
 'BMETH': 'BM-ETH',
 'BOEBTC': 'BOE-BTC',
 'BOEETH': 'BOE-ETH',
 'BOTBTC': 'BOT-BTC',
 'BOTETH': 'BOT-ETH',
 'BRDBTC': 'BRD-BTC',
 'BRDETH': 'BRD-ETH',
 'BRDUSDT': 'BRD-USDT',
 'BTCUSDT': 'BTC-USDT',
 'CGCBTC': 'CGC-BTC',
 'CGCETH': 'CGC-ETH',
 'CGCUSDT': 'CGC-USDT',
 'CMTBTC': 'CMT-BTC',
 'CMTCOB': 'CMT-COB',
 'CMTETH': 'CMT-ETH',
 'CMTUSDT': 'CMT-USDT',
 'COBBTC': 'COB-BTC',
 'COBETH': 'COB-ETH',
 'COBUSDT': 'COB-USDT',
 'COFIBTC': 'COFI-BTC',
 'COFIETH': 'COFI-ETH',
 'CVCBTC': 'CVC-BTC',
 'CVCETH': 'CVC-ETH',
 'DADIBTC': 'DADI-BTC',
 'DADIETH': 'DADI-ETH',
 'DENTBTC': 'DENT-BTC',
 'DENTETH': 'DENT-ETH',
 'DGDBTC': 'DGD-BTC',
 'DGDETH': 'DGD-ETH',
 'DRGNBTC': 'DRGN-BTC',
 'DRGNET

#### Get assset balance

In [44]:
float([balance for balance in cob.wallet.get_balances()['result']['balances'] if balance['currency'] == 'USDT'][0]['total'])

2018-08-10 07:32:53 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.wallet fetch "get_balances"


2.4501111324

In [43]:
[balance for balance in cob.wallet.get_balances()['result']['balances'] if balance['currency'] == 'USDT']

2018-08-10 07:30:33 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.wallet fetch "get_balances"


[{'currency': 'USDT',
  'type': 'exchange',
  'total': '2.4501111324',
  'on_order': '0',
  'locked': False,
  'usd_value': '2.45615722851946754976',
  'btc_value': '0.000383736405556488'}]

In [50]:
cob.market.get_stats()['result']

2018-08-10 07:37:55 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_stats"


{'ABT-BTC': {'id': 'ABT-BTC',
  'last_price': '0.00003',
  'lowest_ask': '0.000046',
  'highest_bid': '0.00002664',
  'base_volume': '223.29',
  'quote_volume': '0.0079407135611545',
  'is_frozen': False,
  'high_24hr': '0.00003566',
  'low_24hr': '0.00003',
  'percent_changed_24hr': '-0.1587212563095906'},
 'ABT-ETH': {'id': 'ABT-ETH',
  'last_price': '0',
  'lowest_ask': '0.000749',
  'highest_bid': '0.0005905',
  'base_volume': '0',
  'quote_volume': '0',
  'is_frozen': False,
  'high_24hr': '0',
  'low_24hr': '0',
  'percent_changed_24hr': '0'},
 'ACT-BTC': {'id': 'ACT-BTC',
  'last_price': '0.00000777',
  'lowest_ask': '0',
  'highest_bid': '0',
  'base_volume': '549',
  'quote_volume': '0.00426573',
  'is_frozen': False,
  'high_24hr': '0.00000777',
  'low_24hr': '0.00000777',
  'percent_changed_24hr': '0'},
 'BAT-BTC': {'id': 'BAT-BTC',
  'last_price': '0',
  'lowest_ask': '0.00004361',
  'highest_bid': '0.0000347',
  'base_volume': '0',
  'quote_volume': '0',
  'is_frozen': Fal

In [56]:
float([data['last_price'] for coin_pair, data in cob.market.get_stats()['result'].items() if data['id'] == 'BTC-USDT'][0])

2018-08-10 07:41:37 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_stats"


6366.6